In [17]:
# hermex_assistant_chatbot.py
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
# from langchian
import asyncio, nest_asyncio
nest_asyncio.apply()

# loading AI models
llm = ChatOllama(model='llama3.2:latest', temperature=0)
embed = OllamaEmbeddings(model='mxbai-embed-large:latest')

# File path for persistent FAISS index
FAISS_PATH = "hermex_faiss_index"

# URLs to load
page_urls = [
    "https://hermextravels.com/",
    "https://hermextravels.com/features.html",
    "https://hermextravels.com/contact.html",
    "https://hermextravels.com/investors.html",
    "https://hermextravels.com/investors.html#investment-tiers"
]


# Setup or load FAISS vectorstore
def setup_vectorstore():
    if os.path.exists(FAISS_PATH):
        print("🔁 Loading FAISS index from disk...")
        return FAISS.load_local(FAISS_PATH, embed, allow_dangerous_deserialization=True)
    else:
        print("🔨 Creating FAISS index from webpages...")
        loader = WebBaseLoader(web_paths=page_urls)
        docs = loader.load()
        splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        chunks = splitter.split_documents(docs)
        vectorstore = FAISS.from_documents(chunks, embed)
        vectorstore.save_local(FAISS_PATH)
        return vectorstore

# Retriever called
retriever = setup_vectorstore().as_retriever(search_kwargs={"k": 2})


# The Hera Assistant
class HermexAssistant:
    def __init__(self)->None:
        load_dotenv()
        self.llm = llm
        self.embedding_model = embed
        self.prompt_template = ChatPromptTemplate.from_template(
            """
            You are Hera, a professional and efficient travel assistant for Hermex Travels.
            Your purpose is to provide accurate, concise, and helpful answers strictly based on the provided context.
            Avoid speculation and only provide facts relevant to Hermex services and the travel/tourism industry.

            Use the following context to answer the user's question. If the answer is not in the context, reply with:
            "I'm sorry, I do not have information on that at the moment."

            Context:
            {context}

            Question: {question}

            Answer as a knowledgeable, polite, and articulate HUMAN travel assistant.
            """
        )
        self.retriever = retriever
        self.qa_chain = self._setup_qa_chain()
        self.user_sessions: dict = {}


    def _setup_qa_chain(self)-> RetrievalQA:
        return RetrievalQA.from_chain_type(
            llm=self.llm,
            retriever=self.retriever,
            return_source_documents=False,
             chain_type="stuff"
        )

    # chat endpoint
    def answer_question(self, user_id: str, query: str) -> str:
        if query.strip(" "):
            return self._standard_answer(user_id, query)

    # normal response
    def _standard_answer(self, user_id: str, query: str)-> str:
        if user_id not in self.user_sessions:
            self.user_sessions[user_id] = []

        result = self.qa_chain.run(query)
        self.user_sessions[user_id].append((query, result))
        return result

    # get user history
    def get_chat_history(self, user_id: str):
        return self.user_sessions.get(user_id, [])

    # clear use history
    def clear_chat_history(self, user_id: str):
        if user_id in self.user_sessions:
            del self.user_sessions[user_id]
            return True
        return False



🔨 Creating FAISS index from webpages...


In [18]:
hera = HermexAssistant()
hera.answer_question("user1", "What is Hermex Travels?")

'Hermex Travels appears to be a travel company that is currently in development and is integrating various technologies such as AI, Augmented Reality (AR), and Cryptocurrency into its services. The company aims to revolutionize the travel experience with its innovative features, including personalized travel planning, AR navigation, and cryptocurrency rewards and payments.'

In [19]:
hera.answer_question("user1", "Who are you?")

"I'm an AI designed to assist and communicate with users. I don't have a personal identity, but I can provide information and answer questions based on my training data. It appears that the text you provided is a list of phone numbers, possibly for countries or regions around the world. If you'd like to know more about any specific number or country, feel free to ask!"